In [1]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import imblearn
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from collections import Counter
#from dask.dataframe import from_pandas
#import dask.dataframe as dd
training_val_combined=pd.read_parquet('combined_sampled/training.gzip', engine='auto')
training_val_combined=training_val_combined.reset_index()
training_val_combined=training_val_combined[['measurements','labels']]
training_val_combined['labels']=training_val_combined['labels'].astype(int)

In [27]:
from sklearn.preprocessing import OneHotEncoder
# define one hot encoding

In [32]:
X=np.vstack(training_val_combined['measurements'])
y=training_val_combined['labels']
cro=Counter(y)
print(cro)
y=np.vstack(y)
org1=list(cro.items())
org=list(cro.items())
org=[(j,i) for i,j in org]
org.sort()
new=[(j,((max(cro.values())-i))) for i,j in org]
new.sort()
new=dict(new)
new[0]=max(cro.values())
over_samp = imblearn.over_sampling.RandomOverSampler(sampling_strategy=new)
X_balanced,y_balanced=over_samp.fit_resample(X,y)

Counter({0: 1006926, 3: 206436, 2: 73068, 1: 55216})


In [33]:
idx = np.random.permutation(len(X_balanced))
X_balanced_reshaped = X_balanced[idx].reshape(len(X_balanced),3,2,1)
y_balanced_reshaped = y_balanced[idx].reshape(-1,1)

In [4]:
num_classes=len(np.unique(y_balanced_reshaped))

In [ ]:
num_classes

In [34]:
y_balanced_reshaped.shape

(3692984, 1)

In [42]:
encoder = OneHotEncoder()
# transform data
onehot = encoder.fit_transform(y_balanced_reshaped)
y_balanced_reshaped_onehot=onehot.toarray()

In [43]:
y_balanced_reshaped_onehot

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [23]:
def make_model(input_shape,num_classes=num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=64, kernel_size=2, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling2D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="relu")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_balanced_reshaped.shape[1:],num_classes=num_classes)
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [24]:
model = make_model(input_shape=X_balanced_reshaped.shape[1:],num_classes=num_classes)
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [26]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 3, 2, 1)]         0         
                                                                 
 conv2d_24 (Conv2D)          (None, 3, 2, 64)          320       
                                                                 
 batch_normalization_24 (Bat  (None, 3, 2, 64)         256       
 chNormalization)                                                
                                                                 
 re_lu_24 (ReLU)             (None, 3, 2, 64)          0         
                                                                 
 conv2d_25 (Conv2D)          (None, 3, 2, 64)          16448     
                                                                 
 batch_normalization_25 (Bat  (None, 3, 2, 64)         256       
 chNormalization)                                          

In [45]:

history = model.fit(
    X_balanced_reshaped,
    y_balanced_reshaped_onehot,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500


2023-04-17 20:22:16.538111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-17 20:22:16.990101: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17aead690
2023-04-17 20:22:16.990128: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17aead690
2023-04-17 20:22:16.993332: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17aead690
2023-04-17 20:22:16.993351: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x17aead690
2023-04-17 20:22:16.997650: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOU

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_12' defined at (most recent call last):
    File "/Users/mfarooq2/miniconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/mfarooq2/miniconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/mfarooq2/miniconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/mfarooq2/miniconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/mfarooq2/miniconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/mfarooq2/miniconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/mfarooq2/miniconda3/li

In [ ]:
X_balanced.shape[1:]

In [ ]:
input_shape=X_balanced_reshaped.shape[1:]

In [ ]:
input_layer = keras.layers.Input(input_shape)

In [ ]:
input_layer.shape